In [1]:

import pandas as pd
import numpy as np
!pip install ace_tools
import itertools

from simuloop_module import simuloop_1 as simuloop
from simuhete import simuhete
from scipy.optimize import dual_annealing

In [3]:
def generate_random_prob(rowX=2742, k=6):
    num_structures = 2 ** k  # 2^k 通りの参入構造

    # ランダムな非負値を生成してから、行ごとに正規化（合計1）
    raw = np.random.rand(rowX, num_structures)
    prob = raw / raw.sum(axis=1, keepdims=True)

    return prob

# 例: 使用
prob = generate_random_prob()



In [4]:
#誤差項の生成、一度シンプルにmarket-とfirmのみです
epsi_firm = np.random.normal(0,4,m*k*r).reshape(m,k*r)
epsi_market = np.random.normal(0,4,m*r).reshape(m,r)
epsi_market = np.repeat(epsi_market, repeats=k, axis=1)

epsi = epsi_firm + epsi_market

In [7]:
#初期値の読み込み
param_lines = []
with open('/Users/user/Desktop/replication/5368_data and programs_0/start_values.txt') as f:
    sim_num = str(int(f.readline().strip()))  # 1行目：sim_num
    for line in f:
        param_lines.extend([float(x) for x in line.strip().split()])
#データの読み込み
datairline = pd.read_stata("/Users/user/Desktop/replication/5368_data and programs_0/CilibertoTamerEconometrica.dta").set_index("market")

#企業数とシミュレーション数、outcomeの組み合わせ数を定義
k = 6
r = 100
total = 2**k


#説明変数と目的変数の獲得
X = datairline.iloc[:, k:]
y = datairline.iloc[:, :k] 

#マーケット数とカラム数の獲得          
rowX = m = X.shape[0] 
colX  = X.shape[1]

#それぞれの市場毎に全ての組み合わせを作る。
##(0,1)^6
index = list(itertools.product([0, 1], repeat=k))
index = np.array(index)  
##(0,1)^6を市場分だけ引き延ばす
repindex = np.tile(index, (X.shape[0], 1))

#CCPを持ってくる
#prob = np.loadtxt('conddensityorder.raw')

#誤差項の追加
epsi_firm = np.random.normal(0,4,m*k*r).reshape(m,k*r)
epsi_market = np.random.normal(0,4,m*r).reshape(m,r)
epsi_market = np.repeat(epsi_market, repeats=k, axis=1)

epsi = epsi_firm + epsi_market

#シミュレーション
iteration = 1
param0 = np.array(param_lines)
oldresultsasa = np.concatenate(([0, 0, iteration, r], param0))

df = pd.DataFrame([oldresultsasa])
log_path = f'oldresultsasa{sim_num}.csv'
df.to_csv(log_path, index=False)

bounds = [(-30, 30)] * len(param0)

result = dual_annealing(
    simuhete,
    bounds=bounds,
    x0=param0,  # ← 初期値をここで指定
    args=(datairline, prob, repindex, k, r, rowX, oldresultsasa, iteration, log_path)
)
param = result.x
fval = result.fun



KeyboardInterrupt: 

In [9]:
simuhete(
    datairline,      # データ
    param0,          # パラメータ（例: 最適化対象）
    prob,            # 確率行列など
    repindex,        # リピートインデックス
    k,               # パラメータの次元数？
    r,               # シミュレーション回数？
    rowX,            # 特徴量 or 入力データ
    oldresultsasa,   # 前回の結果を格納した配列
    iteration,       # 現在のイテレーション番号
    log_path         # ログ出力パス
)

42.139191601952895

In [44]:
oldresultsasa

array([  0.    ,   0.    ,   1.    , 100.    ,   1.3714,   0.5686,
        -0.6776,   0.2736,   0.2042,   0.2962,   0.983 , -13.3598,
        -0.3217,  12.131 ,  -0.7806,  -9.5423,  -9.0561,  -6.5715,
       -10.2389, -15.4918, -11.9097])

In [12]:
firmmarketerrs = np.loadtxt('firmmarketerrs.raw')

epsi_firm = firmmarketerrs[:, :r*k]
epsi_market = firmmarketerrs[:, r*k:r*k+r]
epsi_market = np.kron(epsi_market, np.ones((1, k)))

epsi = epsi_firm + epsi_market

# 空港誤差の合算
AirRndErr = datairline[:, k+colX:k+colX+2*r]
epsiairport = AirRndErr[:, :r] + AirRndErr[:, r:2*r]
epsiairport = np.kron(epsiairport, np.ones((1, k)))

epsi += epsiairport

(64, 6)

In [1085]:
import pandas as pd
import numpy as np
k=6
r=100


datairline = pd.read_stata("/Users/user/Desktop/replication/5368_data and programs_0/CilibertoTamerEconometrica.dta").set_index("market")
coldatairline = datairline.shape[1]
X = datairline.iloc[:, k:]

colX = X.shape[1]               
y = datairline.iloc[:, :k]           
rowX = X.shape[0] 

param =[-1,1,1,1,1,1,1,1,1,1,1,-3,-3,-3,-3,-3,-3]
constant = param[0]  # 定数項
paramX = param[1:9]      # 外生変数
paraheterog1 = param[9]  # 異質性1
paraheterog2 = param[10] # 異質性2
parafirm1 = param[11:17] # 競合効果


roleX = X.iloc[:, :8] @ paramX
roleX = np.tile(roleX.to_numpy().reshape(-1, 1), (1, k))

roleheter = X.iloc[:, 8:14].to_numpy() * paraheterog1
roleheter += X.iloc[:, 14:20].to_numpy() * paraheterog2

index = list(itertools.product([0, 1], repeat=k))
index = np.array(index)  # ⬅️ NumPy配列に変換
total = index.shape[0]


onothereffect = np.zeros((total * X.shape[0], k))
repindex = np.tile(index, (X.shape[0], 1))

for i in range(k):
    effect = repindex[:, i:i+1] * parafirm1[i]
    effect = np.tile(effect, (1, k))
    effect[:, i] = 0  # 自分自身の影響は除外
    onothereffect += effect

owneffect = np.ones((total, k)) * constant


repi_roleX = np.repeat(roleX, repeats=2 ** k, axis=0)


repi_roleheter = np.repeat(roleheter, repeats=2 ** k, axis=0)


repi_owneffect = np.repeat(owneffect, repeats=rowX, axis=0)



common = repi_owneffect + repi_roleheter + repi_owneffect


In [679]:
sumupp = np.zeros((rowX, total))
sumlow = np.zeros((rowX, total))

totcount = r * np.ones((rowX, 1))
temp = np.zeros((rowX, 1))

In [680]:
epsitemp = np.random.normal(loc=0.0, scale=0.4, size=rowX * k).reshape(rowX, k)
epsitemp = np.repeat(epsitemp, repeats=2 ** k, axis=0)
epsitemp = epsitemp*repindex

equil = epsitemp + common  
equil = ((equil>=0).astype(int) == repindex).astype(int)
sumequil = np.sum(equil, axis=1)

In [952]:
# vectorequil.shape = (rowX * total, 1)
cumsumvectorequil = np.cumsum(vectorequil)

# 各市場の構造セットの末尾（totalごと）を取得
sumvectorequil = cumsumvectorequil[total - 1::total].reshape(-1, 1)

# 差分で「各市場ごとの均衡数」を得る
if sumvectorequil.shape[0] > 1:
    sumvectorequil[1:] = np.diff(sumvectorequil, axis=0)


In [1107]:
r = 100
k = 6

datairline = pd.read_stata("/Users/user/Desktop/replication/5368_data and programs_0/CilibertoTamerEconometrica.dta").set_index("market")
coldatairline = datairline.shape[1]
X = datairline.iloc[:, k:]

colX = X.shape[1]               
y = datairline.iloc[:, :k]           
rowX = X.shape[0] 

param =[-1,1,1,1,1,1,1,1,1,1,1,-2,-5,-5,-5,-3,-3]

paraconstant = param[0]
paramX = param[1:9]
paraheterog1 = param[9]
paraheterog2 = param[10]
parafirm1 = param[11:17]

roleX = X.iloc[:, :8] @ paramX
roleX = np.tile(roleX.to_numpy().reshape(-1, 1), (1, k))

roleheter = X.iloc[:, 8:14].to_numpy() * paraheterog1
roleheter += X.iloc[:, 14:20].to_numpy() * paraheterog2

index = list(itertools.product([0, 1], repeat=k))
index = np.array(index)  # ⬅️ NumPy配列に変換
total = index.shape[0]


onothereffect = np.zeros((total * X.shape[0], k))
repindex = np.tile(index, (X.shape[0], 1))

for i in range(k):
    effect = repindex[:, i:i+1] * parafirm1[i]
    effect = np.tile(effect, (1, k))
    effect[:, i] = 0  # 自分自身の影響は除外
    onothereffect += effect


owneffect = np.ones((total, k)) * constant


repi_roleX = np.repeat(roleX, repeats=2 ** k, axis=0)


repi_roleheter = np.repeat(roleheter, repeats=2 ** k, axis=0)


repi_owneffect = np.repeat(owneffect, repeats=rowX, axis=0)



common = repi_owneffect + repi_roleheter + onothereffect + repi_roleX


m=rowX
epsi = np.random.normal(0,1,m*k*r).reshape(m,k*r)
meanlow, meanupp, temp = simuloop(common,m,k,r,epsi,repindex)

sand_matrix = np.where(
    prob[:rowX, :] > meanupp,
    (prob[:rowX, :] - meanupp)**2,
    0
    ) + np.where(
    prob[:rowX, :] < meanlow,
    (prob[:rowX, :] - meanlow)**2,
    0
    )


neverpure = np.min(np.isfinite(meanupp), axis=1)
findneverpure = np.where(neverpure == 0)[0]
if findneverpure.size > 0:
    sand_matrix = np.delete(sand_matrix, findneverpure, axis=0)

    # (8b) 最終的な目的関数値

sand

69.46187614097005

In [1109]:
temp

0.0

In [1087]:
sand

2712.7261549440022

In [1088]:
epsi

array([[-0.01337112, -0.16941508, -0.79899802, ..., -0.57495291,
        -1.65754941, -1.76166082],
       [ 0.10309189,  0.50729191,  0.02501056, ..., -2.36426709,
        -0.88372689, -0.03261999],
       [ 1.19242533, -0.54538449,  0.95097319, ...,  0.0913684 ,
        -0.42829933, -1.27909086],
       ...,
       [-0.4325977 , -1.86133972, -0.71048022, ..., -0.13740077,
        -0.64740327, -0.05958329],
       [-0.86538209,  0.55569474, -0.29900997, ..., -0.82036187,
         1.13741549,  0.79065163],
       [ 0.3791659 , -0.13211334, -0.80367836, ..., -0.44456348,
         0.64059484, -1.2335481 ]])

In [1008]:
meanupp.shape

(2742, 64)

In [1033]:
sand_matrix = np.where(
    prob[:rowX, :] > meanupp,
    (prob[:rowX, :] - meanupp)**2,
    0
    ) + np.where(
    prob[:rowX, :] < meanlow,
    (prob[:rowX, :] - meanlow)**2,
    0
    )


'1'

In [1125]:
param0 

array([  1.3714,   0.5686,  -0.6776,   0.2736,   0.2042,   0.2962,
         0.983 , -13.3598,  -0.3217,  12.131 ,  -0.7806,  -9.5423,
        -9.0561,  -6.5715, -10.2389, -15.4918, -11.9097])

In [1040]:
def generate_random_prob(rowX=2742, k=6):
    num_structures = 2 ** k  # 2^k 通りの参入構造

    # ランダムな非負値を生成してから、行ごとに正規化（合計1）
    raw = np.random.rand(rowX, num_structures)
    prob = raw / raw.sum(axis=1, keepdims=True)

    return prob

# 例: 使用
prob = generate_random_prob()
print(prob.shape)  # (100, 8)
print(prob[0]) 

(2742, 64)
[0.00468551 0.00629758 0.00367664 0.012303   0.01877722 0.01240937
 0.02003066 0.02595366 0.02319822 0.0288849  0.00346524 0.02401404
 0.00863344 0.02753008 0.01171728 0.00719037 0.00948059 0.01799681
 0.00423367 0.00169862 0.00240419 0.01318267 0.00371383 0.00757567
 0.00546887 0.01389372 0.01633612 0.02135398 0.00831313 0.02342413
 0.01779609 0.00619948 0.01803567 0.02886598 0.02190796 0.0232041
 0.00446385 0.02197312 0.01332275 0.02314004 0.00055283 0.03036913
 0.0290115  0.02504026 0.02648547 0.01644149 0.01067827 0.00153083
 0.00127077 0.00976511 0.00277184 0.02037731 0.02337788 0.02778734
 0.02359827 0.0273322  0.02223196 0.00827893 0.0098685  0.02942276
 0.02232029 0.01524056 0.02760274 0.02189156]


In [968]:
#2*2 を考える(一変数)
np.random.seed(42) 
X = np.random.exponential(scale=6.0, size=3,)


index = list(itertools.product([0, 1], repeat=2))
index = np.array(index) 
total = index.shape[0]

parameter = [1,2,-10,-10]
repindex = np.tile(index, (X.shape[0], 1))
paramX = parameter[1]
parafirm1 = parameter[2:4]
constant = parameter[0]

roleX = X * paramX
roleX = np.tile(roleX.reshape(-1, 1), (1, 2))



index = list(itertools.product([0, 1], repeat=2))
index = np.array(index)  # ⬅️ NumPy配列に変換


onothereffect = np.zeros((total * X.shape[0], 2))
repindex = np.tile(index, (X.shape[0], 1))

for i in range(2):
    effect = repindex[:, i:i+1] * parafirm1[i]
    effect = np.tile(effect, (1, 2))
    effect[:, i] = 0  # 自分自身の影響は除外
    onothereffect += effect

owneffect = np.ones((total, 2)) * constant


repi_roleX = np.repeat(roleX, repeats=2 ** 2, axis=0)


repi_owneffect = np.repeat(owneffect, repeats=X.shape[0], axis=0)


common = repi_roleX+repi_owneffect+onothereffect

common

array([[ 6.63121708,  6.63121708],
       [-3.36878292,  6.63121708],
       [ 6.63121708, -3.36878292],
       [-3.36878292, -3.36878292],
       [37.12145717, 37.12145717],
       [27.12145717, 37.12145717],
       [37.12145717, 27.12145717],
       [27.12145717, 27.12145717],
       [16.80094832, 16.80094832],
       [ 6.80094832, 16.80094832],
       [16.80094832,  6.80094832],
       [ 6.80094832,  6.80094832]])

In [954]:
r=2
k= 2
m=3
epsi = np.random.normal(0,10,m*k*r).reshape(m,k*r)
X = np.random.exponential(scale=6.0, size=3,)





index = list(itertools.product([0, 1], repeat=2))
index = np.array(index) 
total = index.shape[0]

parameter = [1,2,-10,-10]
repindex = np.tile(index, (X.shape[0], 1))
paramX = parameter[1]
parafirm1 = parameter[2:4]
constant = parameter[0]

roleX = X * paramX
roleX = np.tile(roleX.reshape(-1, 1), (1, 2))



index = list(itertools.product([0, 1], repeat=2))
index = np.array(index)  # ⬅️ NumPy配列に変換


onothereffect = np.zeros((total * X.shape[0], 2))
repindex = np.tile(index, (X.shape[0], 1))



for i in range(2):
    effect = repindex[:, i:i+1] * parafirm1[i]
    effect = np.tile(effect, (1, 2))
    effect[:, i] = 0  # 自分自身の影響は除外
    onothereffect += effect

owneffect = np.ones((total, 2)) * constant


repi_roleX = np.repeat(roleX, repeats=2 ** 2, axis=0)


repi_owneffect = np.repeat(owneffect, repeats=X.shape[0], axis=0)


common = repi_roleX+repi_owneffect+onothereffect


sumupp = np.zeros((rowX, total))
sumlow = np.zeros((rowX, total))

totcount = r * np.ones((rowX, 1))
temp = np.zeros((rowX, 1))


for i in np.arange(r):
    epsitemp = epsi[:, k * i : k * (i + 1)] 
    epsitemp = np.repeat(epsitemp, repeats=2 ** 2, axis=0)
    equil = epsitemp + common  
    equil = ((equil>=0).astype(int) == repindex).astype(int)
    sumequil = np.sum(equil, axis=1)
    vectorequil = (sumequil == 2).astype(int)
    
    # vectorequil.shape = (rowX * total, 1)
    cumsumvectorequil = np.cumsum(vectorequil)

    # 各市場の構造セットの末尾（totalごと）を取得
    sumvectorequil = cumsumvectorequil[total - 1::total].reshape(-1, 1)

    # 差分で「各市場ごとの均衡数」を得る
    if sumvectorequil.shape[0] > 1:
        sumvectorequil[1:] = np.diff(sumvectorequil, axis=0)
    
    upp = vectorequil.reshape(X.shape[0], total)
    low = upp.copy()
    lowtobezero = np.where(sumvectorequil > 1)[0]
    # 該当する市場の行をゼロに
    low[lowtobezero, :] = 0

    upptobezero = np.where(sumvectorequil == 0)[0]

    # 均衡ゼロの市場が存在する場合のみ処理
    if upptobezero.size > 0:
        count = (sumvectorequil == 0).astype(int).reshape(-1, 1)
        temp += count
        totcount -= count

    sumupp += upp
    sumlow += low
    print(upp)
# 平均下限と上限の計算（ブロードキャストにより行方向に割り算）
meanlow = sumlow / totcount  # shape: (rowX, total)
meanupp = sumupp / totcount  # shape: (rowX, total)

# 均衡なし市場の割合（全シミュレーション × 市場数 で割る）
temp = np.sum(temp) / (r * rowX)
meanlow,meanupp

[[0 1 1 0]
 [0 0 1 0]
 [1 0 0 0]]
[[0 1 0 0]
 [0 1 1 0]
 [0 1 0 0]]


(array([[0. , 0.5, 0. , 0. ],
        [0. , 0. , 0.5, 0. ],
        [0.5, 0.5, 0. , 0. ]]),
 array([[0. , 1. , 0.5, 0. ],
        [0. , 0.5, 1. , 0. ],
        [0.5, 0.5, 0. , 0. ]]))

In [957]:
# 平均下限と上限の計算（ブロードキャストにより行方向に割り算）
meanlow = sumlow / totcount  # shape: (rowX, total)
meanupp = sumupp / totcount  # shape: (rowX, total)

# 均衡なし市場の割合（全シミュレーション × 市場数 で割る）
temp = np.sum(temp) / (r * rowX)


In [958]:

upp = vectorequil.reshape(X.shape[0], total)

# 下限は上限のコピーで初期化
low = upp.copy()

# 複数均衡の市場インデックスを検出
lowtobezero = np.where(sumvectorequil > 1)[0]

# 該当する市場の行をゼロに
low[lowtobezero, :] = 0

In [991]:
k ** 2

36

In [989]:
epsitemp = epsi[:, k * 0 : k * (0 + 1)] 
epsitemp = np.repeat(epsitemp, repeats=k ** 2, axis=0)
epsitemp.shape

(98712, 6)

In [1084]:
def simuloop(common,m,k,r,epsi,repindex):
    total=2**k

    #複数均衡数とユニークな均衡をカウントする
    sumupp = np.zeros((m, total))
    sumlow = np.zeros((m, total))

    #上をシミュレーション回数で割ることで割合にするので作成
    totcount = r * np.ones((m, 1))


    #均衡が存在しないケースを考慮し、tempにその情報を保存していく（これを最終的にtotocountから引くことで割合を正当化）
    temp = np.zeros((m, 1))

    #シミュレーションの実行
    for i in np.arange(r):
        ##誤差項加えるパート
        #シミレーションごとに誤差を獲得（積分）。
        epsitemp = epsi[:, k * i : k * (i + 1)] 
        epsitemp = np.repeat(epsitemp, repeats=2 ** k, axis=0)
        #観測できる利潤に加える
        equil = epsitemp + common  

        ##均衡を探すパート
        #近郊では逸脱、もしくは実際のアクションが0以上なら１を取り、それ以外では0をとる。
        #それがrepindexと一致してれば、プレイヤーが最適反応をとっているということ
        equil = ((equil>=0).astype(int) == repindex).astype(int)

        #最適反応をとっているプレイヤー数の獲得
        #それが全体のプレイヤー数と一致するならそれは均衡である。
        #vectorequilにはあるoutcomが均衡かどうかが市場ごとに入っている
        sumequil = np.sum(equil, axis=1)
        vectorequil = (sumequil == k).astype(int)
        
        #各市場における均衡数を獲得する。
        #助長なやり方であるが、累積和を取り、total毎にスライス、その差分を取ることで市場毎の均衡数を獲得。
        cumsumvectorequil = np.cumsum(vectorequil)
        sumvectorequil = cumsumvectorequil[total - 1::total].reshape(-1, 1)
        if sumvectorequil.shape[0] > 1:
            sumvectorequil[1:] = np.diff(sumvectorequil, axis=0)
        
        #upperは複数均衡を考慮してもいいので、そのままカウントする。
        #ただし形は(m, total)で記載していく
        upp = vectorequil.reshape(m, total)

        #lowは複数均衡が発生したらカウントできない。そのため.whereで複数均衡が存在している市場を特定し0にする
        low = upp.copy()
        lowtobezero = np.where(sumvectorequil > 1)[0]
        low[lowtobezero, :] = 0

        ##均衡数が０の対処
        #存在するのであればその数だけ減らす。またtenpにカウントすることで最終的に、均衡が存在しない数を表示できる。
        upptobezero = np.where(sumvectorequil == 0)[0]
        if upptobezero.size > 0:
            count = (sumvectorequil == 0).astype(int).reshape(-1, 1)
            temp += count
            totcount -= count
        
        #シミュレーション毎に加算
        sumupp += upp
        sumlow += low


    # 下限確率と上限確率の計算
    meanlow = sumlow / totcount 
    meanupp = sumupp / totcount  

    # 均衡なし市場の割合
    temp = np.sum(temp) / (r * m)
    return meanlow , meanupp,temp

In [960]:
sumvectorequil

array([[1],
       [2],
       [1]])

In [961]:
sumvectorequil = cumsumvectorequil[total - 1::total].reshape(-1, 1)


In [962]:
# vectorequil.shape = (rowX * total, 1)
cumsumvectorequil = np.cumsum(vectorequil)

# 各市場の構造セットの末尾（totalごと）を取得
sumvectorequil = cumsumvectorequil[total - 1::total].reshape(-1, 1)

# 差分で「各市場ごとの均衡数」を得る
if sumvectorequil.shape[0] > 1:
    sumvectorequil[1:] = np.diff(sumvectorequil, axis=0)

In [785]:
((equil>0).astype(int) == repindex).astype(int)

array([[1, 1],
       [1, 1],
       [1, 1],
       [0, 0],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [0, 1]])

In [322]:
data = pd.read_stata("/Users/user/Desktop/replication/5368_data and programs_0/CilibertoTamerEconometrica.dta")

#記述統計の作成
summry = data.iloc[:,1:].agg(["mean","std"])

#企業特性の作成
firm_specific_columns = {
    'entry': 'airline',
    'airport_presence': 'marketpresence',
    'cost': 'mindistancefromhub'
}

carriers =  ['AA', 'DL', 'UA', 'AL', 'LCC', 'WN']
firm_result = {}

for label,columns in firm_specific_columns.items():
    row = {}
    for carrier in carriers:
        colname = f"{columns}{carrier}"
        mean = summry.at['mean', colname]
        std = summry.at['std', colname]
        row[carrier] = f"{mean:.3f} ({std:.3f})"
    firm_result[label] = row


firm_summary = pd.DataFrame(firm_result).T
firm_summary.index.name = 'Variable'

#市場特性
market_vars = [
    'marketdistance', 'fromcenterdistance',
    'mindistance', 'changeincmarket', 
    'percapitaincmarket', 'marketsize',
    'wrightamendmDAL', 'dallasmarket'
]

market_summary = data[market_vars].agg(['mean', 'std']).T.round(3)
market_summary.columns = ['Mean', 'Std. Dev.']
market_summary.index.name = 'Variable'

print("=== Firm-Level Summary ===")
display(firm_summary)

print("\n=== Market-Level Summary ===")
display(market_summary)

=== Firm-Level Summary ===


,AA,DL,UA,AL,LCC,WN
Variable,,,,,,
entry,0.426 (0.495),0.551 (0.497),0.275 (0.447),0.548 (0.498),0.162 (0.369),0.247 (0.431)
airport_presence,0.422 (0.167),0.540 (0.181),0.265 (0.153),0.376 (0.135),0.098 (0.077),0.242 (0.176)
cost,0.736 (1.609),0.420 (1.322),0.784 (1.476),0.229 (0.615),0.043 (0.174),0.303 (0.860)



=== Market-Level Summary ===


,Mean,Std. Dev.
Variable,,
marketdistance,1.085,0.624
fromcenterdistance,1.571,0.594
mindistance,0.346,0.205
changeincmarket,4.051,1.478
percapitaincmarket,3.240,0.391
marketsize,2.259,1.846
wrightamendmDAL,0.030,0.169
dallasmarket,0.070,0.255


In [331]:
#ノンパラ推定

nonepara_data = pd.DataFrame()
# y: 6社の参入ベクトルをまとめて1つの列に
entry_vars = ['airlineAA', 'airlineDL', 'airlineUA', 'airlineAL', 'airlineLCC', 'airlineWN']
nonepara_data['entry_vector'] = data[entry_vars].apply(tuple, axis=1)


# 離散化する変数(mindistancefromhubLCCだけ、0が多いのでerrorが起きるため除く)
continue_vars = [
    'marketdistance', 'fromcenterdistance', 'mindistance', 'changeincmarket',
    'percapitaincmarket', 'marketsize', 
    'marketpresenceAA', 'marketpresenceDL', 'marketpresenceUA',
    'marketpresenceAL', 'marketpresenceLCC', 'marketpresenceWN',
    'mindistancefromhubAA', 'mindistancefromhubDL', 'mindistancefromhubUA',
    'mindistancefromhubAL',  'mindistancefromhubWN'
]

# 離散化（4分位に分けてカテゴリ化）
for var in continue_vars:
    try:
        # 分位点の bin を先に確認
        quantile_bins = pd.qcut(data[var], q=4, retbins=True, duplicates='drop')[1]
        num_bins = len(quantile_bins) - 1

        if num_bins < 2:
            print(f"{var}: 分位点の数が少なすぎて離散化できません。スキップします。")
            continue

        labels = list(range(1, num_bins + 1))
        new_col = f"{var}_q"
        nonepara_data[new_col] = pd.qcut(data[var], q=num_bins, labels=labels, duplicates='drop')

    except Exception as e:
        print(f"{var} でエラー: {e}")


#mindistancefromhubLCCだけ、0が多いのでerrorが起きるため,0なら0それ以外で3分位
is_zero = data["mindistancefromhubLCC"] == 0
non_zero_vals = data.loc[~is_zero, "mindistancefromhubLCC"]
quantiles = pd.qcut(non_zero_vals, q=3, labels=[1, 2, 3], duplicates='drop')
nonepara_data["mindistancefromhubLCC_q"] = 0
nonepara_data.loc[~is_zero,"mindistancefromhubLCC_q"] = quantiles 

#indexをmarkeにして、離散化変数を追加
nonepara_data.index = data["market"]
nonepara_data[["wrightamendmDAL","dallasmarket"]] = data[["wrightamendmDAL","dallasmarket"]].values

In [332]:
# クロス集計: x_state × entry_vector の頻度表
freq_table = data.groupby(['x_state', 'entry_vector']).size().reset_index(name='count')

# 各 x_state における条件付き確率（割合）を計算
freq_table['prob'] = freq_table.groupby('x_state')['count'].transform(lambda x: x / x.sum())


KeyError: 'x_state'